### Sponsor / MVNO change for subscribers in DMI

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from tqdm import tqdm
from importlib import reload
from os.path import join
import re

import roamability as rb

In [35]:
#######################################################################################
# Define variables here
#######################################################################################

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
out_file = 'toc_sponsor.rcjson'

mvno_dict = {'TECHMOB':'TECHMOB_ESIM'}
sponsor_dict = {'MB_ALL':'MB_Tottolli'}

# Ranges
ranges = """
454030227306300-454030227306674,
454030227319000-454030227319999,
454030227615000-454030227615999,
454030227616000-454030227616999,
454030227617000-454030227617999
"""

#######################################################################################

#######################################################################################

In [36]:
# Define Class

class ImsiRange:

    def __init__(self, ranges, mvno_dict, sponsor_dict):
        self.range_list = self.parse_ranges(ranges)
        self.mvno_dict = mvno_dict
        self.sponsor_dict = sponsor_dict
    
    def parse_ranges(self, ranges):
        """Convert ranges to list"""
        pattern_clean = re.compile('([\n\s])')
        ranges = re.sub(pattern_clean, '', ranges)
        pattern_range = re.compile(r'(\d{15})\D+(\d{15})')
        range_list = []
        matches = pattern_range.finditer(ranges)
        for match in matches:
            range_list.append([match.group(1),match.group(2)])
        return range_list
    
    def prepare_range_list_for_sql(self):
        """Convert ranges in the list to string for SQL"""
        str_range_list_for_sql = ''
        for i, ranges in enumerate(imsi_range.range_list):
            str_range_list_for_sql = str_range_list_for_sql + f'(IMSI BETWEEN {ranges[0]} AND {ranges[1]})'
            if i+1 < len(self):
                str_range_list_for_sql = str_range_list_for_sql + ' OR '
        return str_range_list_for_sql
        
    def check_sponsor(self):
        sql_str = """
SELECT
SUBSTR(s.IMSI,1,12) AS IMSI_RANGE,
SUBSTR(si.IMSI_NUMBER,1,12) AS S_IMSI_RANGE,
sp.NAME AS PROFILE_NAME,
COUNT(*) AS NUM
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND si.SUBSCRIBER_REF IN
(
SELECT RI FROM SUBSCRIBER WHERE
{}
)
GROUP BY SUBSTR(s.IMSI,1,12),SUBSTR(si.IMSI_NUMBER,1,12),sp.NAME
ORDER BY SUBSTR(s.IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_sponsor_gr)
        
    def get_data_for_sponsor_change(self):
        sql_str = """
SELECT
si.CLI_MSISDN,s.IMSI,si.PROF_ID,si.IMSI_NUMBER,si.flag_confirmed,sp.NAME
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND
({})
ORDER BY s.IMSI,si.PROF_ID""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_sponsor
    
    def prepare_toc_to_cange_sponsor(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_sponsor_change()
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        df_dmi_imsi.NAME.replace(to_replace=self.sponsor_dict, inplace=True)
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.s_imsi.modify,CLI_MSISDN="'
        t2='",FLAG_CONFIRMED="0",IMSI_NUMBER="'
        t3='",PROF_ID="'
        t4='",SPONSOR_REF="'
        t5='",SUBSCRIBER_REF="'
        t6='";'
        for i, [cli, imsi, pid, simsi, sponsor] in enumerate(
            df_dmi_imsi[['CLI_MSISDN','IMSI','PROF_ID','IMSI_NUMBER','NAME']].values):
            ouf.write(t1+str(cli)+t2+str(simsi)+t3+str(pid)+t4+str(sponsor)+t5+str(imsi)+t6+'\n')
        ouf.close()
    
    def __len__(self):
        return len(self.range_list)
    
    def __str__(self):
        """Print ranges"""
        for i, ranges in enumerate(self.range_list):
            print(f'{i+1:02} : {ranges[0]} - {ranges[1]} -> {1+int(ranges[1])-int(ranges[0])}')
        return ''
    
imsi_range = ImsiRange(ranges, mvno_dict, sponsor_dict)

print(imsi_range)

01 : 454030227306300 - 454030227306674 -> 375
02 : 454030227319000 - 454030227319999 -> 1000
03 : 454030227615000 - 454030227615999 -> 1000
04 : 454030227616000 - 454030227616999 -> 1000
05 : 454030227617000 - 454030227617999 -> 1000



In [40]:
# Check Sponsors in DMI for the given ranges

imsi_range.check_sponsor()

,IMSI_RANGE,S_IMSI_RANGE,PROFILE_NAME,NUM
0,454030227306,454030227306,MB_Tottolli,375
1,454030227319,454030227319,MB_Tottolli,1000
2,454030227615,454030227615,MB_Tottolli,1000
3,454030227616,454030227616,MB_Tottolli,1000
4,454030227617,454030227617,MB_Tottolli,1000


In [38]:
# Prepare TOC commands to change the Sponsor in DMI

imsi_range.prepare_toc_to_cange_sponsor(downloads, out_file)

,NAME,IMSI
0,MB_ALL,4375


,NAME,IMSI
0,MB_Tottolli,4375


,CLI_MSISDN,IMSI,PROF_ID,IMSI_NUMBER,FLAG_CONFIRMED,NAME
4372,8529750107997,454030227617997,1,454030227617997, ,MB_Tottolli
4373,8529750107998,454030227617998,1,454030227617998, ,MB_Tottolli
4374,8529750107999,454030227617999,1,454030227617999, ,MB_Tottolli


### Developments

In [2]:
reseller_name = 'Tottolli-Tech-Mobil'
presision = 12 # Digits in IMSI

sql_srt = """
SELECT RESELLER_NAME,account_name,SUBSTRING(IMSI,1,{1}) AS IMSI_RANGE,COUNT(*) AS NUM
,MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM
(
SELECT
r.RESELLER_NAME
,a.account_name
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
--AND ss.status = 'Active'
AND r.RESELLER_NAME = '{0}'
) t
GROUP BY RESELLER_NAME,account_name,SUBSTRING(IMSI,1,{1})
ORDER BY SUBSTRING(IMSI,1,{1}),account_name
""".format(reseller_name, presision)
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs_rslr_rng = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_rslr_rng

,RESELLER_NAME,account_name,IMSI_RANGE,NUM,MIN_IMSI,MAX_IMSI
0,Tottolli-Tech-Mobil,Tottolli S8 Soft,234500026400,20,234500026400002,234500026400021
1,Tottolli-Tech-Mobil,Tottolli S8 Soft,234500026425,1000,234500026425000,234500026425999
2,Tottolli-Tech-Mobil,Tottolli S8 Soft,234500026426,550,234500026426000,234500026426549
3,Tottolli-Tech-Mobil,Tottolli S8 Soft,234500026427,1000,234500026427000,234500026427999
4,Tottolli-Tech-Mobil,Tottolli S8 Soft,234500026428,1000,234500026428000,234500026428999
...,...,...,...,...,...,...
70,Tottolli-Tech-Mobil,Tottolli S5 Soft,515030191003,1000,515030191003000,515030191003999
71,Tottolli-Tech-Mobil,Tottolli S5 Soft,515030191004,1000,515030191004000,515030191004999
72,Tottolli-Tech-Mobil,Tottolli S5 Soft,515030191005,1000,515030191005000,515030191005999
73,Tottolli-Tech-Mobil,Tottolli S5 Soft,515030191006,1000,515030191006000,515030191006999


In [30]:
df_ocs_rslr_rng.loc[df_ocs_rslr_rng.IMSI_RANGE.str.startswith('454')]

,RESELLER_NAME,account_name,IMSI_RANGE,NUM,MIN_IMSI,MAX_IMSI
62,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227305,195,454030227305145,454030227305349
63,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227306,375,454030227306300,454030227306674
64,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227319,1000,454030227319000,454030227319999
65,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227615,1000,454030227615000,454030227615999
66,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227616,1000,454030227616000,454030227616999
67,Tottolli-Tech-Mobil,Tottolli S4 Soft,454030227617,1000,454030227617000,454030227617999


In [31]:
for imsi in df_ocs_rslr_rng.loc[df_ocs_rslr_rng.IMSI_RANGE.str.startswith('454'), ['MIN_IMSI','MAX_IMSI']].values:
    print(f'{imsi[0]}-{imsi[1]},')

454030227305145-454030227305349,
454030227306300-454030227306674,
454030227319000-454030227319999,
454030227615000-454030227615999,
454030227616000-454030227616999,
454030227617000-454030227617999,
